In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained("dpv/finetuned-gpt2-tiny")
model = GPT2LMHeadModel.from_pretrained("dpv/finetuned-gpt2-tiny")

/Users/fabio/miniconda3/envs/pydvl_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.4M parameters


In [3]:
mlp_model_size = sum(t.numel() for name, t in model.named_parameters() if "mlp" in name)
print(f"Mlp parameters size: {mlp_model_size/1000**2:.1f}M parameters")

Mlp parameters size: 56.7M parameters


In [4]:
for name, t in model.named_parameters():
    if "mlp" not in name:
        t.requires_grad = False

In [5]:
for name, t in model.named_parameters():
    print(name)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

## Dataset

In [6]:
text = "In a near future, in a little human colony, the human race"
encoded_input = tokenizer(text, return_tensors="pt")
# output_base = model(**encoded_input)
output = model.generate(
    **encoded_input, max_length=25, pad_token_id=tokenizer.eos_token_id
)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [7]:
print(output_text)

In a near future, in a little human colony, the human race will be able to create a new species of life.


In [8]:
output

tensor([[  818,   257,  1474,  2003,    11,   287,   257,  1310,  1692, 18815,
            11,   262,  1692,  3234,   481,   307,  1498,   284,  2251,   257,
           649,  4693,   286,  1204,    13]])

# Directly with model

In [9]:
encoded_input = tokenizer(text, return_tensors="pt")
model_output = model(**encoded_input, labels=encoded_input["input_ids"])
tokenizer.decode(model_output.logits.argmax(dim=2)[0])

' the statement-, the the world over-, the human race will'

In [10]:
next_token_logits = model_output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
tokenizer.decode(next_token, skip_special_tokens=True)

' will'

In [11]:
torch.argmax(model_output.logits, dim=-1)

tensor([[ 262, 2643,   12,   11,  262,  262,  995,  625,   12,   11,  262, 1692,
         3234,  481]])

In [12]:
tokenizer.decode(
    torch.argmax(model_output.logits, dim=-1)[0],
    skip_special_tokens=True,
)

' the statement-, the the world over-, the human race will'

In [13]:
def batch_sentence(sentence_tokens, embedding_dim=50257):
    inputs_ids = []
    label = []
    for label_token_idx in range(1, len(sentence_tokens)):
        input_ids = [tokenizer.eos_token_id] * (
            len(sentence_tokens) - 1 - label_token_idx
        ) + sentence_tokens[:label_token_idx]
        inputs_ids.append(input_ids)
        label.append(sentence_tokens[label_token_idx])
    return torch.as_tensor(inputs_ids), torch.as_tensor(label)

In [14]:
len_input = len(encoded_input["input_ids"][0])
batch_input_tokens, labels = batch_sentence(output[0].tolist())

In [15]:
model_output = model(batch_input_tokens)
next_token_logits = model_output.logits[:, -1, :]

In [16]:
next_tokens = torch.argmax(model_output.logits[:, -1, :], dim=-1)
tokenizer.decode(
    next_tokens,
    skip_special_tokens=True,
)

' the recent-, the which world over-, a world race will be able to create a new species of life.'

### compare batch_input to model output with labels

In [17]:
text = "In "
encoded_input = tokenizer(text, return_tensors="pt")
# output_base = model(**encoded_input)
output = model.generate(
    **encoded_input, max_length=25, pad_token_id=tokenizer.eos_token_id
)

In [18]:
batch_input_tokens, labels = batch_sentence(output[0].tolist())

In [19]:
next_token_logits = []
for input_ in batch_input_tokens:
    model_output = model(torch.as_tensor(input_))
    next_token_logits.append(model_output.logits[-1, :])

# Dealing with train dataset

In [20]:
train_dataset = load_dataset("roneneldan/TinyStories", split="train")

train_ds = train_dataset.shuffle().select(range(1))

Using custom data configuration roneneldan--TinyStories-6ac769f186d7da53
Found cached dataset parquet (/Users/fabio/.cache/huggingface/datasets/roneneldan___parquet/roneneldan--TinyStories-6ac769f186d7da53/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [21]:
from datasets import Dataset as HFDataset


def gen():
    yield {"text": output_text}


test_ds = HFDataset.from_generator(gen)

Using custom data configuration default-574ae6c787a6bd21
Found cached dataset generator (/Users/fabio/.cache/huggingface/datasets/generator/default-574ae6c787a6bd21/0.0.0)


In [22]:
from torch.utils.data import DataLoader, Dataset


class TinyStoriesDataset(Dataset):
    def __init__(self, dataset, tokenizer, input_len=None):
        self.tokenized_ds = dataset.map(
            lambda x: tokenizer(x["text"], return_tensors="pt")
        )
        if input_len is None:
            self.input_len = (
                min([len(x["input_ids"][0]) for x in self.tokenized_ds]) - 1
            )
        else:
            self.input_len = input_len
        self.last_sentence_id = None
        self.last_batched_sentence = None

    def __len__(self):
        return len(self.tokenized_ds) * (self.input_len - 1)

    def __getitem__(self, idx):
        sentence_id = idx // self.input_len
        if sentence_id != self.last_sentence_id:
            self.last_sentence_id = sentence_id
            self.last_batched_sentence, self.sentence_labels = batch_sentence(
                self.tokenized_ds[sentence_id]["input_ids"][0][: self.input_len + 1]
            )
        return (
            self.last_batched_sentence[idx % self.input_len],
            self.sentence_labels[idx % self.input_len],
        )

In [23]:
batched_train_ds = TinyStoriesDataset(train_ds, tokenizer, 3)
batched_test_ds = TinyStoriesDataset(test_ds, tokenizer, 3)

100%|██████████| 1/1 [00:00<00:00, 400.79ex/s]


In [24]:
train_dataloader = DataLoader(batched_train_ds, batch_size=3)
test_dataloader = DataLoader(batched_test_ds, batch_size=3)

In [28]:
class PatchedGPT2LMHeadModel(GPT2LMHeadModel):
    class PatchedGPT2LMHeadModel(GPT2LMHeadModel):
        def __init__(self, config):
            super().__init__(config)
            self.conv1d = torch.nn.Conv1d(
                config.hidden_size, config.hidden_size, kernel_size=3, padding=1
            )

        def forward(self, input_ids, **kwargs):
            outputs = super().forward(input_ids, **kwargs)
            hidden_states = outputs[0]
            hidden_states = hidden_states.transpose(
                1, 2
            )  # Transpose to (batch_size, hidden_size, sequence_length)
            conv_output = self.conv1d(hidden_states)
            conv_output = conv_output.transpose(
                1, 2
            )  # Transpose back to (batch_size, sequence_length, hidden_size)
            logits = self.lm_head(conv_output)
            return logits[:, -1, :]

In [29]:
from pydvl.influence import compute_influences
from pydvl.influence.torch import TorchTwiceDifferentiable

model = PatchedGPT2LMHeadModel.from_pretrained("dpv/finetuned-gpt2-tiny")

ekfac_train_influences = compute_influences(
    TorchTwiceDifferentiable(model, torch.nn.functional.cross_entropy),
    training_data=train_dataloader,
    test_data=test_dataloader,
    influence_type="up",
    inversion_method="ekfac",
    hessian_regularization=0.1,
    on_layers=["lm_head"],
    progress=True,
)
mean_ekfac_train_influences = np.mean(ekfac_train_influences.numpy(), axis=0)

Batch Test Gradients: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([2, 3]) torch.Size([2])
